### Creating Input-Target PAIRS

In [1]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [2]:
with open("Data/the-verdict.txt","r",encoding='utf-8') as f:
    raw_text = f.read()

In [3]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [8]:
enc_text = enc_text[50:]

In [9]:
context_size = 4 #length of the input


x = enc_text[:context_size]
y = enc_text[1:context_size+1]

print(f"x: {x}")

print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [11]:
for i in range(1,context_size+1):
    context = enc_text[:i]
    desired = enc_text[i]

    print(context , "----->" , desired)

[290] -----> 4920
[290, 4920] -----> 2241
[290, 4920, 2241] -----> 287
[290, 4920, 2241, 287] -----> 257


In [13]:
for i in range(1,context_size+1):
    context = enc_text[:i]
    desired = enc_text[i]

    print(tokenizer.decode(context) , "----->" , tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


### IMPLEMENTING A DATA LOADER

In [ ]:
from torch.utils.data import Dataset,DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,txt, tokenizer,max_length, stride):
        self.input_ids = []
        self.target_ids = []


        token_ids = tokenizer.encode(txt, allowed_special={"|<endoftext>|"})


        for i in range(0,len(token_ids)-max_length,stride):
            input_chunks = token_ids[i:i+max_length]
            target_chunks = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunks))
            self.target_ids.append(torch.tensor(target_chunks))

    
    def __len__(self):
        return len(self.input_ids)
    

    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx] # type: ignore

## Creating Data Loader

In [29]:
def create_dataloader_v1(text,batch_size=4,max_length =256,
                        stride = 128, shuffle = True, drop_last = True,
                        num_workers = 0):


    tokenizer = tiktoken.get_encoding('gpt2')

    # create dataset
    dataset = GPTDatasetV1(text, tokenizer,max_length, stride)

    # create dataloader

    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )

    return dataloader

In [30]:
with open("Data/the-verdict.txt","r",encoding='utf-8') as f:
    raw_text = f.read()

In [38]:
dataloader = create_dataloader_v1(
    raw_text,batch_size = 1, max_length = 4, stride = 1, shuffle = False
)


data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [39]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


### Effect of batch size

In [41]:
dataloader = create_dataloader_v1(
    raw_text,batch_size = 8, max_length = 4, stride = 4, shuffle = False
)

data_iter = iter(dataloader)

first_input , first_output = next(data_iter)


print("First input ------->")
print(first_input)
print("First output ------->")
print(first_output)

First input ------->
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
First output ------->
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
